In [1]:
import numpy as np
import pandas as pd
import collections 
data = pd.read_csv("train.csv")
data.head()

,Date,Temp9am,Temp3pm,MinTemp,MaxTemp,Rainfall,RainToday,Evaporation,Sunshine,WindGustDir,...,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,ID
0,01-02-13,20.7,20.9,19.5,22.4,15.6,Yes,6.2,0.0,W,...,SSW,17,20,92,84,1017.6,1017.4,8,8,1
1,02-02-13,22.4,24.8,19.5,25.6,6.0,Yes,3.4,2.7,W,...,E,9,13,83,73,1017.9,1016.4,7,7,2
2,03-02-13,23.5,23.0,21.6,24.5,6.6,Yes,2.4,0.1,W,...,ESE,17,2,88,86,1016.7,1015.6,7,8,3
3,04-02-13,21.4,20.9,20.2,22.8,18.8,Yes,2.2,0.0,W,...,E,22,20,83,90,1014.2,1011.8,8,8,4
4,05-02-13,22.5,25.5,19.7,25.7,77.4,Yes,4.8,0.0,W,...,W,11,6,88,74,1008.3,1004.8,8,8,5


In [2]:
print(data.shape)
for item in data.columns:
    print(item)

(2616, 22)
Date
Temp9am
Temp3pm
MinTemp
MaxTemp
Rainfall
RainToday
Evaporation
Sunshine
WindGustDir
WindGustSpeed
WindDir9am
WindDir3pm
WindSpeed9am
WindSpeed3pm
Humidity9am
Humidity3pm
Pressure9am
Pressure3pm
Cloud9am
Cloud3pm
ID


In [3]:
null_columns = []
for item in data.columns:

    if data[item].isnull().any():
        null_columns.append(item)

print(null_columns)
#No null columns :(

[]


In [4]:
data.columns = [
    'date',
    'temp_9am',
    'temp_3pm',
    'min_temp',
    'max_temp',
    'rain_fall',
    'rain_today',
    'evaporation',
    'sunshine',
    'wind_gust_dir',
    'wind_gust_speed',
    'wind_dir_9am',
    'wind_dir_3pm',
    'wind_speed_9am',
    'wind_speed_3pm',
    'humidity_9am',
    'humidity3pm',
    'pressure_9am',
    'pressure_3pm',
    'cloud_9am',
    'cloud_3pm',
    'id'
]
data = data.drop('id' , axis = 1)
data.columns

Index(['date', 'temp_9am', 'temp_3pm', 'min_temp', 'max_temp', 'rain_fall',
       'rain_today', 'evaporation', 'sunshine', 'wind_gust_dir',
       'wind_gust_speed', 'wind_dir_9am', 'wind_dir_3pm', 'wind_speed_9am',
       'wind_speed_3pm', 'humidity_9am', 'humidity3pm', 'pressure_9am',
       'pressure_3pm', 'cloud_9am', 'cloud_3pm'],
      dtype='object')

In [5]:
X = data.drop('rain_today' , axis = 1).copy()
Y = data['rain_today'].copy()

In [6]:
from datetime import datetime

def date_to_days(date : str) -> int:

    time = datetime.strptime(date , "%d-%m-%y")
    ref_from = datetime(2000 , 1 , 1)
    days = (time - ref_from).days

    return days

X['date'] = X['date'].map(date_to_days)
X.head()

,date,temp_9am,temp_3pm,min_temp,max_temp,rain_fall,evaporation,sunshine,wind_gust_dir,wind_gust_speed,wind_dir_9am,wind_dir_3pm,wind_speed_9am,wind_speed_3pm,humidity_9am,humidity3pm,pressure_9am,pressure_3pm,cloud_9am,cloud_3pm
0,4780,20.7,20.9,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,17,20,92,84,1017.6,1017.4,8,8
1,4781,22.4,24.8,19.5,25.6,6.0,3.4,2.7,W,41,W,E,9,13,83,73,1017.9,1016.4,7,7
2,4782,23.5,23.0,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,17,2,88,86,1016.7,1015.6,7,8
3,4783,21.4,20.9,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,22,20,83,90,1014.2,1011.8,8,8
4,4784,22.5,25.5,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,11,6,88,74,1008.3,1004.8,8,8


In [7]:
#We will going to store catagorical columns for CatBoost
cat_columns = []

for item in X.columns:

    if X[item].dtype == object:
        cat_columns.append(item)

for item in cat_columns: print(item)

wind_gust_dir
wind_dir_9am
wind_dir_3pm


In [8]:
for item in cat_columns:
    print(f"{item} : {X[item].unique()}")
    print(f"len : {len(X[item].unique())}")

wind_gust_dir : ['W' 'NNW' 'WNW' 'ENE' 'NNE' 'NW' 'SSE' 'NE' 'ESE' 'WSW' 'SE' 'SW' 'N' 'E'
 'SSW' 'S']
len : 16
wind_dir_9am : ['S' 'W' 'ESE' 'NNE' 'SSW' 'WNW' 'N' 'SW' 'SE' 'SSE' 'WSW' 'E' 'ENE' 'NW'
 'NNW' 'NE']
len : 16
wind_dir_3pm : ['SSW' 'E' 'ESE' 'W' 'ENE' 'S' 'SE' 'SSE' 'NE' 'NNE' 'NNW' 'NW' 'WNW' 'N'
 'WSW' 'SW']
len : 16


In [9]:
from sklearn.preprocessing import OneHotEncoder
#We also store a copy of data for catboost before it is hot encoded.
X_cat_copy = X.copy()

encoder = OneHotEncoder(sparse_output = False , 
                        categories = 'auto', 
                        handle_unknown= 'error',
                        max_categories = 32)

encoder.fit(X[cat_columns])
cat_ohed = pd.DataFrame(encoder.transform(X[cat_columns]))

In [10]:
X = X.drop(cat_columns , axis = 1)
X_encoded = pd.concat([X , cat_ohed] , axis = 1)
X_encoded.head()

,date,temp_9am,temp_3pm,min_temp,max_temp,rain_fall,evaporation,sunshine,wind_gust_speed,wind_speed_9am,...,38,39,40,41,42,43,44,45,46,47
0,4780,20.7,20.9,19.5,22.4,15.6,6.2,0.0,41,17,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,4781,22.4,24.8,19.5,25.6,6.0,3.4,2.7,41,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4782,23.5,23.0,21.6,24.5,6.6,2.4,0.1,41,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4783,21.4,20.9,20.2,22.8,18.8,2.2,0.0,41,22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4784,22.5,25.5,19.7,25.7,77.4,4.8,0.0,41,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
Y_MAPED: pd.Series = Y.map({'Yes' : 1 , 'No' : 0})
Y_MAPED

0       1
1       1
2       1
3       1
4       1
       ..
2611    0
2612    0
2613    0
2614    0
2615    0
Name: rain_today, Length: 2616, dtype: int64

In [12]:
print(Y_MAPED.sum(axis = 0)/len(Y_MAPED))
#So , obviously we need careful stratification before train_test_split()

0.25840978593272174


In [13]:
#Now before feature engineering ,first I want to reduce the number of
# dimensions of X
from sklearn.decomposition import PCA

params_pca: dict = {
    'n_components' : 0.95,
    'whiten'       : False,
    'random_state' : 12,
    'svd_solver'   : 'full',
}

pca_model = PCA(**params_pca)
pca_model.fit(X)
X_reduced_raw_values: np.ndarray = pca_model.transform(X)



In [14]:
col_len: int = X_reduced_raw_values.shape[1]
new_columns: list = [f"PCA_FEATURE_{idx}".lower() for idx in range(col_len)]
print(new_columns)
X_reduced: pd.DataFrame = pd.DataFrame(data = X_reduced_raw_values , columns = new_columns)
X_reduced

['pca_feature_0']


,pca_feature_0
0,-1307.567090
1,-1306.544648
2,-1305.572877
3,-1304.565725
4,-1303.554320
...,...
2611,1303.489310
2612,1304.545363
2613,1305.540950
2614,1306.536836


In [15]:
#Uhhh , it seems we have lost too much data . Even though , we captured
#95% of the total variance, the 1 single dimension is very worse.
#We should ignore that.
from sklearn.model_selection import train_test_split

X_ROOT_TRAIN: pd.DataFrame
X_ROOT_TEST: pd.DataFrame
Y_ROOT_TRAIN: pd.DataFrame
Y_ROOT_TEST: pd.DataFrame

tts_params: dict = {
    'test_size' : float(0.2),
    'train_size' : float(0.8),
    'random_state' : int(13),
    'shuffle' : bool(True),
    'stratify'  : Y_MAPED,
}

X_ROOT_TRAIN , X_ROOT_TEST, Y_ROOT_TRAIN , Y_ROOT_TEST = train_test_split(
    X_encoded,
    Y_MAPED,
    **tts_params
)



In [16]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import optuna

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
class enum_model:
    def __init__(self , model_name: str = "XGBM") -> any:
        if not model_name in ["XGBM" , "CB" , "LGBM"]:
            raise ValueError(
                "Error! Model has to be: [XGBM , CB , LGBM]")
        self.model: str = model_name
    def getModelType(self) -> str:
        return self.model
    

def _obj_for_xgbm(trial: optuna.Trial) -> float:

    xgbm_params: dict = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'tree_method': 'auto',
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
    'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.7, 1.0),
    'colsample_bynode': trial.suggest_float('colsample_bynode', 0.7, 1.0),
    'gamma': trial.suggest_float('gamma', 0.3, 1),
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.09),
    'max_depth': trial.suggest_int('max_depth', 3, 8),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
    'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
    'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 1.5),
    'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 1.5),
    'subsample': trial.suggest_float('subsample', 0.8, 1.0),
    'scale_pos_weight': (Y_ROOT_TRAIN == 0).sum()/(Y_ROOT_TRAIN == 1).sum(),
    'random_state': 13,
    'n_jobs': -1,
    'verbosity': 1
    }


    __model: XGBClassifier = XGBClassifier(**xgbm_params)
    __model.fit(X_ROOT_TRAIN , Y_ROOT_TRAIN )
    validator: StratifiedKFold = StratifiedKFold(n_splits = 10 , 
                                                  shuffle = True , 
                                                  random_state = 13)
    score: float = cross_val_score(__model, 
                                   X_ROOT_TEST, 
                                   Y_ROOT_TEST,
                                   scoring = "accuracy",
                                   n_jobs = -1,
                                   verbose =  0,
                                   cv = validator).mean()
    return score

def _obj_for_lgbm(trial: optuna.Trial) -> float:
     lgbm_params: dict = {
        'boosting_type': 'gbdt',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3 , 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 0.1 , 0.325),
        'max_depth': trial.suggest_int('max_depth', 3 , 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 8),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.3, 0.8),
        'n_estimators': trial.suggest_int('n_estimators', 500 , 1500),
        'n_jobs': -1,
        'objective' : 'binary',
        'max_bin': trial.suggest_int('max_bin', 32 , 64),
        'random_state': 13,
        'scale_pos_weight' : (Y_ROOT_TRAIN == 0).count()/(Y_ROOT_TRAIN == 1).count(),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1 , 1.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1 , 1.5),
        'subsample': trial.suggest_float('subsample', 0.8 , 1),
     }

     __model: LGBMClassifier = LGBMClassifier(**lgbm_params)
     __model.fit(X_ROOT_TRAIN , Y_ROOT_TRAIN)
     validator: StratifiedKFold = StratifiedKFold(n_splits = 10 , 
                                                  shuffle = True , 
                                                  random_state = 13)
     score: float = cross_val_score(__model, 
                                    X_ROOT_TEST, 
                                           Y_ROOT_TEST,
                                           scoring = "accuracy",
                                           n_jobs = -1,
                                           verbose =  0,
                                           cv = validator).mean()
     return score
def _obj_for_cb(trial: optuna.Trial) -> float:
    
    cb_params: dict = {
        'iterations': trial.suggest_int('iterations', 100 , 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.1),
        'depth': trial.suggest_int('depth', 3, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.001, 0.05),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.5, 3),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 5),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 0, 3),
        'verbose': 0,
        'auto_class_weights' : "Balanced",
        'random_state': 13,
        'task_type': 'CPU',  
    }

    model: CatBoostClassifier = CatBoostClassifier(**cb_params)
    validator: StratifiedKFold = StratifiedKFold(
        n_splits=10, 
        shuffle=True, 
        random_state=13
        )
    score = cross_val_score(
        model,
        X_ROOT_TEST,
        Y_ROOT_TEST,
        scoring='accuracy',
        cv=validator,
        n_jobs=-1,
        verbose=0
    ).mean()
    
    return score

class func:
    pass
def objective(model_name: enum_model) -> func:
    match(model_name.getModelType()):
        case "XGBM": return _obj_for_xgbm
        case "LGBM": return _obj_for_lgbm
        case "CB"  : return _obj_for_cb

In [18]:
xgbm_obj: func = objective(name := enum_model("XGBM"))

storage_path_xgbm: str = "sqlite:///xgbm_study.db"
study_xgbm: optuna.Study = optuna.study.create_study(
    study_name = "xgbm_study",
    storage  = storage_path_xgbm,
    direction= 'maximize',
    )
study_xgbm.optimize(
    func = xgbm_obj,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = True,
)


[I 2025-12-24 17:16:17,897] A new study created in RDB with name: xgbm_study
Best trial: 4. Best value: 0.799746:   2%|▏         | 1/50 [00:09<07:40,  9.39s/it]

[I 2025-12-24 17:16:27,272] Trial 4 finished with value: 0.7997460087082728 and parameters: {'colsample_bytree': 0.8540516660248922, 'colsample_bylevel': 0.8848851722266012, 'colsample_bynode': 0.8502898069296946, 'gamma': 0.8758251265146406, 'learning_rate': 0.06458362204352769, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 749, 'reg_alpha': 1.3341877036674663, 'reg_lambda': 0.9449823505406738, 'subsample': 0.9711549918213684}. Best is trial 4 with value: 0.7997460087082728.


Best trial: 5. Best value: 0.822351:   4%|▍         | 2/50 [00:11<03:53,  4.87s/it]

[I 2025-12-24 17:16:28,958] Trial 5 finished with value: 0.8223512336719885 and parameters: {'colsample_bytree': 0.9284873214724647, 'colsample_bylevel': 0.9450074309066967, 'colsample_bynode': 0.7986918436905629, 'gamma': 0.8078112124922427, 'learning_rate': 0.07789985846966926, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 1108, 'reg_alpha': 0.8915777229052616, 'reg_lambda': 0.6374524524663514, 'subsample': 0.8327693033803022}. Best is trial 5 with value: 0.8223512336719885.


Best trial: 5. Best value: 0.822351:   6%|▌         | 3/50 [00:12<02:42,  3.45s/it]

...

[I 2025-12-24 17:17:42,674] Trial 49 finished with value: 0.8321480406386067 and parameters: {'colsample_bytree': 0.8022625910595685, 'colsample_bylevel': 0.7319550073876682, 'colsample_bynode': 0.8599696075067734, 'gamma': 0.46858769510621207, 'learning_rate': 0.03209634515623573, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 572, 'reg_alpha': 0.9912455916326537, 'reg_lambda': 1.2346243391670542, 'subsample': 0.9189526900735641}. Best is trial 43 with value: 0.8398403483309144.


Best trial: 43. Best value: 0.83984:  98%|█████████▊| 49/50 [01:25<00:00,  1.04it/s]

[I 2025-12-24 17:17:43,185] Trial 48 finished with value: 0.8245645863570392 and parameters: {'colsample_bytree': 0.8102343070139064, 'colsample_bylevel': 0.7343756143860348, 'colsample_bynode': 0.8700471112895833, 'gamma': 0.4566852134873728, 'learning_rate': 0.033879692471962645, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 585, 'reg_alpha': 0.9776764580177145, 'reg_lambda': 1.2445304148795662, 'subsample': 0.9209556416232723}. Best is trial 43 with value: 0.8398403483309144.


Best trial: 43. Best value: 0.83984: 100%|██████████| 50/50 [01:25<00:00,  1.71s/it]

[I 2025-12-24 17:17:43,604] Trial 47 finished with value: 0.8322206095791002 and parameters: {'colsample_bytree': 0.8208144975609676, 'colsample_bylevel': 0.7405150097770261, 'colsample_bynode': 0.8625194917795111, 'gamma': 0.4127235558826793, 'learning_rate': 0.03395210760824832, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 695, 'reg_alpha': 0.9828233818307274, 'reg_lambda': 1.2299991181265777, 'subsample': 0.922555763665309}. Best is trial 43 with value: 0.8398403483309144.


In [19]:
lgbm_obj: func = objective(name := enum_model("LGBM"))

storage_path_lgbm: str = "sqlite:///lgbm_study.db"
study_lgbm = optuna.create_study(
    study_name = "lgbm_study",
    storage  = storage_path_lgbm,
    direction= 'maximize',
)
study_lgbm.optimize(
    func = lgbm_obj,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = True 
)

[I 2025-12-24 17:17:43,783] A new study created in RDB with name: lgbm_study
Best trial: 5. Best value: 0.830152:   2%|▏         | 1/50 [00:02<02:08,  2.63s/it]

[I 2025-12-24 17:17:46,401] Trial 5 finished with value: 0.8301523947750363 and parameters: {'colsample_bytree': 0.7502860871756024, 'learning_rate': 0.2196975754143041, 'max_depth': 6, 'min_child_samples': 4, 'min_child_weight': 0.6505291404665022, 'n_estimators': 583, 'max_bin': 64, 'reg_alpha': 1.451187816295349, 'reg_lambda': 1.1215633899614212, 'subsample': 0.9533506640963598}. Best is trial 5 with value: 0.8301523947750363.


Best trial: 5. Best value: 0.830152:   4%|▍         | 2/50 [00:04<01:41,  2.11s/it]

[I 2025-12-24 17:17:48,141] Trial 4 finished with value: 0.8148040638606677 and parameters: {'colsample_bytree': 0.4682924225826799, 'learning_rate': 0.31077550759195716, 'max_depth': 4, 'min_child_samples': 4, 'min_child_weight': 0.3946601497441891, 'n_estimators': 1139, 'max_bin': 39, 'reg_alpha': 0.9682903974017226, 'reg_lambda': 0.1434145761574826, 'subsample': 0.8945718089420679}. Best is trial 5 with value: 0.8301523947750363.


Best trial: 5. Best value: 0.830152:   6%|▌         | 3/50 [00:04<01:02,  1.34s/it]

...

[I 2025-12-24 17:18:26,113] Trial 47 finished with value: 0.8090348330914366 and parameters: {'colsample_bytree': 0.6153494317564142, 'learning_rate': 0.23497750284247662, 'max_depth': 5, 'min_child_samples': 5, 'min_child_weight': 0.6925548314244467, 'n_estimators': 502, 'max_bin': 41, 'reg_alpha': 0.3881288807423415, 'reg_lambda': 1.1249715669881346, 'subsample': 0.9490169781927426}. Best is trial 29 with value: 0.8340348330914369.


Best trial: 29. Best value: 0.834035:  98%|█████████▊| 49/50 [00:42<00:00,  2.10it/s]

[I 2025-12-24 17:18:26,538] Trial 49 finished with value: 0.8167997097242381 and parameters: {'colsample_bytree': 0.8999076288751692, 'learning_rate': 0.2844761655472373, 'max_depth': 5, 'min_child_samples': 5, 'min_child_weight': 0.6838940882089496, 'n_estimators': 548, 'max_bin': 41, 'reg_alpha': 0.44659483351646173, 'reg_lambda': 1.1143571360094868, 'subsample': 0.9515600240826875}. Best is trial 29 with value: 0.8340348330914369.


Best trial: 29. Best value: 0.834035: 100%|██████████| 50/50 [00:43<00:00,  1.16it/s]

[I 2025-12-24 17:18:26,883] Trial 48 finished with value: 0.8149129172714078 and parameters: {'colsample_bytree': 0.882847883117095, 'learning_rate': 0.23309828333745358, 'max_depth': 5, 'min_child_samples': 5, 'min_child_weight': 0.6882195779133069, 'n_estimators': 715, 'max_bin': 42, 'reg_alpha': 0.4520351178776989, 'reg_lambda': 1.0982355477447183, 'subsample': 0.9525931935371491}. Best is trial 29 with value: 0.8340348330914369.


In [20]:
cb_obj: func = objective(enum_model("CB"))
storage_path_cb: str = "sqlite:///cb_study.db"
study_cb = optuna.create_study(
    study_name = "cb_study",
    storage  = storage_path_cb,
    direction= 'maximize',
    load_if_exists = True
)
study_cb.optimize(
    func = cb_obj,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = True 
)

[I 2025-12-24 17:18:27,095] A new study created in RDB with name: cb_study
Best trial: 4. Best value: 0.816691:   2%|▏         | 1/50 [00:03<02:33,  3.14s/it]

[I 2025-12-24 17:18:30,213] Trial 4 finished with value: 0.8166908563134978 and parameters: {'iterations': 103, 'learning_rate': 0.08749867369835852, 'depth': 8, 'l2_leaf_reg': 0.024239446762694832, 'bagging_temperature': 0.9303067239358558, 'random_strength': 1.1918790395503627, 'min_data_in_leaf': 3, 'subsample': 0.8567946035310015, 'max_ctr_complexity': 2}. Best is trial 4 with value: 0.8166908563134978.


Best trial: 4. Best value: 0.816691:   4%|▍         | 2/50 [00:09<03:49,  4.78s/it]

[I 2025-12-24 17:18:36,144] Trial 3 finished with value: 0.813134978229318 and parameters: {'iterations': 458, 'learning_rate': 0.06758313974466292, 'depth': 7, 'l2_leaf_reg': 0.009379452972731399, 'bagging_temperature': 0.4524924852652302, 'random_strength': 2.852317642855379, 'min_data_in_leaf': 2, 'subsample': 0.8554376415835598, 'max_ctr_complexity': 3}. Best is trial 4 with value: 0.8166908563134978.


Best trial: 4. Best value: 0.816691:   6%|▌         | 3/50 [00:10<02:35,  3.32s/it]

...

[I 2025-12-24 17:20:58,455] Trial 47 finished with value: 0.80177793904209 and parameters: {'iterations': 328, 'learning_rate': 0.09258268873873418, 'depth': 3, 'l2_leaf_reg': 0.016416498952427833, 'bagging_temperature': 0.07009727589488698, 'random_strength': 1.4200129657544132, 'min_data_in_leaf': 5, 'subsample': 0.7682910433663271, 'max_ctr_complexity': 1}. Best is trial 7 with value: 0.8284470246734399.


Best trial: 7. Best value: 0.828447:  98%|█████████▊| 49/50 [02:32<00:01,  1.85s/it]

[I 2025-12-24 17:20:59,867] Trial 48 finished with value: 0.8149854862119013 and parameters: {'iterations': 311, 'learning_rate': 0.09179452854367451, 'depth': 3, 'l2_leaf_reg': 0.016669812098487106, 'bagging_temperature': 0.05669583461423544, 'random_strength': 1.4827973645198105, 'min_data_in_leaf': 5, 'subsample': 0.7729144328521381, 'max_ctr_complexity': 1}. Best is trial 7 with value: 0.8284470246734399.


Best trial: 7. Best value: 0.828447: 100%|██████████| 50/50 [02:33<00:00,  3.07s/it]

[I 2025-12-24 17:21:00,451] Trial 49 finished with value: 0.8017053701015966 and parameters: {'iterations': 318, 'learning_rate': 0.06501031818457506, 'depth': 3, 'l2_leaf_reg': 0.016814201314583267, 'bagging_temperature': 0.17686468947066045, 'random_strength': 1.429568537643279, 'min_data_in_leaf': 5, 'subsample': 0.7704140626875751, 'max_ctr_complexity': 1}. Best is trial 7 with value: 0.8284470246734399.


In [21]:
best_params_xgbm: dict = study_xgbm.best_params.copy()
BASE_MODEL_XGBM: XGBClassifier = XGBClassifier(**best_params_xgbm)
BASE_MODEL_XGBM.fit(X_ROOT_TRAIN , 
                    Y_ROOT_TRAIN , 
                    eval_set = [(X_ROOT_TRAIN , Y_ROOT_TRAIN)],
                    verbose = True)

[0]	validation_0-logloss:0.55683
[1]	validation_0-logloss:0.54385
[2]	validation_0-logloss:0.53159
[3]	validation_0-logloss:0.51947
[4]	validation_0-logloss:0.50910
[5]	validation_0-logloss:0.49862
[6]	validation_0-logloss:0.48878
[7]	validation_0-logloss:0.47934
[8]	validation_0-logloss:0.47065
[9]	validation_0-logloss:0.46213
[10]	validation_0-logloss:0.45464
[11]	validation_0-logloss:0.44706
[12]	validation_0-logloss:0.44004
[13]	validation_0-logloss:0.43331
[14]	validation_0-logloss:0.42695
[15]	validation_0-logloss:0.42035
[16]	validation_0-logloss:0.41476
[17]	validation_0-logloss:0.40982
[18]	validation_0-logloss:0.40484
[19]	validation_0-logloss:0.39924
[20]	validation_0-logloss:0.39365
[21]	validation_0-logloss:0.38892
[22]	validation_0-logloss:0.38387
[23]	validation_0-logloss:0.37970
[24]	validation_0-logloss:0.37548
[25]	validation_0-logloss:0.37163
[26]	validation_0-logloss:0.36792
[27]	validation_0-logloss:0.36414
[28]	validation_0-logloss:0.36034
[29]	validation_0-loglos

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,0.7495492658880007
,colsample_bynode,0.8892921611009232
,colsample_bytree,0.8361439915378459
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [22]:
best_params_lgbm: dict = study_lgbm.best_params.copy()
BASE_MODEL_LGBM: LGBMClassifier = LGBMClassifier(objective = "binary" ,**best_params_lgbm)
BASE_MODEL_LGBM.fit(
    X_ROOT_TRAIN,
    Y_ROOT_TRAIN,
    eval_set = [(X_ROOT_TEST, Y_ROOT_TEST)],
)

[LightGBM] [Info] Number of positive: 541, number of negative: 1551
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 579
[LightGBM] [Info] Number of data points in the train set: 2092, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258604 -> initscore=-1.053236
[LightGBM] [Info] Start training from score -1.053236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,4
,learning_rate,0.2555283957191829
,n_estimators,940
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.614155083502968
,min_child_samples,7


In [23]:
best_params_cb: dict = study_cb.best_params.copy()
BASE_CB_MODEL: CatBoostClassifier = CatBoostClassifier(**best_params_cb)
BASE_CB_MODEL.fit(
    X_ROOT_TRAIN,
    Y_ROOT_TRAIN,
    eval_set = [(X_ROOT_TEST , Y_ROOT_TEST)],
    verbose = 1
)

0:	learn: 0.6409374	test: 0.6459891	best: 0.6459891 (0)	total: 147ms	remaining: 37.5s
1:	learn: 0.6044806	test: 0.6104927	best: 0.6104927 (1)	total: 149ms	remaining: 18.9s
2:	learn: 0.5653541	test: 0.5737205	best: 0.5737205 (2)	total: 150ms	remaining: 12.7s
3:	learn: 0.5365016	test: 0.5480593	best: 0.5480593 (3)	total: 152ms	remaining: 9.56s
4:	learn: 0.5119464	test: 0.5245942	best: 0.5245942 (4)	total: 153ms	remaining: 7.69s
5:	learn: 0.4906127	test: 0.5048682	best: 0.5048682 (5)	total: 155ms	remaining: 6.44s
6:	learn: 0.4726090	test: 0.4895176	best: 0.4895176 (6)	total: 156ms	remaining: 5.55s
7:	learn: 0.4606911	test: 0.4784869	best: 0.4784869 (7)	total: 158ms	remaining: 4.88s
8:	learn: 0.4485360	test: 0.4676951	best: 0.4676951 (8)	total: 159ms	remaining: 4.36s
9:	learn: 0.4334020	test: 0.4535918	best: 0.4535918 (9)	total: 160ms	remaining: 3.94s
10:	learn: 0.4240406	test: 0.4464062	best: 0.4464062 (10)	total: 162ms	remaining: 3.6s
11:	learn: 0.4131571	test: 0.4384049	best: 0.4384049 

In [24]:
xgbm_meta_pred: np.ndarray = BASE_MODEL_XGBM.predict_proba(X_ROOT_TEST)
lgbm_meta_pred: np.ndarray = BASE_MODEL_LGBM.predict_proba(X_ROOT_TEST)
cb_meta_pred: np.ndarray   = BASE_CB_MODEL.predict_proba(X_ROOT_TEST)

In [25]:
df_meta: pd.DataFrame = pd.DataFrame({
    'XGBM_PREDICTION' : xgbm_meta_pred[:, 1],
    'LGBM_PREDICTION' : lgbm_meta_pred[:, 1],
    'CB_PREDICTION'   : cb_meta_pred[:, 1],
    'ACTUAL_VALUE'    : Y_ROOT_TEST
})
df_meta

,XGBM_PREDICTION,LGBM_PREDICTION,CB_PREDICTION,ACTUAL_VALUE
127,0.767872,0.963614,0.587927,1
2504,0.011376,0.000279,0.055879,0
634,0.061866,0.071258,0.053363,0
1797,0.499557,0.624508,0.293189,1
2153,0.059738,0.003185,0.191931,0
...,...,...,...,...
880,0.697241,0.386192,0.678978,0
2031,0.009570,0.000009,0.011525,0
1600,0.073930,0.008644,0.080184,0
1581,0.017920,0.000866,0.049671,0


In [26]:
from sklearn.ensemble import GradientBoostingClassifier

META_X = df_meta.drop('ACTUAL_VALUE' , axis = 1).copy()
META_Y = df_meta['ACTUAL_VALUE']

META_X_TRAIN , META_X_TEST, META_Y_TRAIN , META_Y_TEST = train_test_split(
    META_X,
    META_Y,
    train_size = 0.5,
    test_size  = 0.5,
    stratify = META_Y,
    shuffle = True,
    random_state = 13
)
def obj_for_meta(trial: optuna.Trial) -> float:

    gbm_params: dict = {
        'loss' : 'log_loss',
        'learning_rate' : trial.suggest_float('learning_rate', 0.1 , 0.15),
        'n_estimators'  : trial.suggest_int('n_estimators', 30, 100),
        'subsample'     : trial.suggest_float('subsample', 0.8 , 1),
        'min_samples_split': int(2),
        'min_samples_leaf' : int(1),
        'max_depth'      : trial.suggest_int('max_depth', 3, 8),
        'random_state' : int(13)
    }

    __model: GradientBoostingClassifier = GradientBoostingClassifier(
        **gbm_params
    )
    __model.fit(META_X_TRAIN , META_Y_TRAIN)
    validator: StratifiedKFold = StratifiedKFold(
        n_splits= 10 ,shuffle= True, random_state= 13)
    accuracy: float = cross_val_score(
        __model,
        META_X_TEST,
        META_Y_TEST,
        cv = validator,
        scoring = "accuracy",
        n_jobs = -1,
    ).mean()
    return accuracy

storage_path_gbm: str = "sqlite:///gbm_study.db"
study_gbm = optuna.create_study(
    study_name = "gbm_study_5",
    storage  = storage_path_gbm,
    direction= 'maximize',
    load_if_exists = True
)
study_gbm.optimize(
    func = obj_for_meta,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = True 
)

[I 2025-12-24 17:21:04,558] A new study created in RDB with name: gbm_study_5
Best trial: 4. Best value: 0.794017:   2%|▏         | 1/50 [00:01<00:55,  1.13s/it]

[I 2025-12-24 17:21:05,565] Trial 4 finished with value: 0.794017094017094 and parameters: {'learning_rate': 0.11145964130833638, 'n_estimators': 55, 'subsample': 0.9917387717588708, 'max_depth': 3}. Best is trial 4 with value: 0.794017094017094.


Best trial: 4. Best value: 0.794017:   2%|▏         | 1/50 [00:01<00:55,  1.13s/it]

[I 2025-12-24 17:21:06,218] Trial 2 finished with value: 0.8132478632478632 and parameters: {'learning_rate': 0.13207309340036216, 'n_estimators': 31, 'subsample': 0.8870897857825569, 'max_depth': 3}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-24 17:21:06,283] Trial 1 finished with value: 0.7903133903133902 and parameters: {'learning_rate': 0.1219841750717819, 'n_estimators': 97, 'subsample': 0.9697295297178214, 'max_depth': 4}. Best is trial 2 with value: 0.8132478632478632.


Best trial: 2. Best value: 0.813248:  10%|█         | 5/50 [00:02<00:14,  3.09it/s]

...

[I 2025-12-24 17:21:14,672] Trial 43 finished with value: 0.794159544159544 and parameters: {'learning_rate': 0.12207946173540486, 'n_estimators': 75, 'subsample': 0.9399976178598788, 'max_depth': 4}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-24 17:21:14,724] Trial 45 finished with value: 0.7978632478632478 and parameters: {'learning_rate': 0.12288678103380551, 'n_estimators': 35, 'subsample': 0.9396558999266422, 'max_depth': 4}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-24 17:21:14,774] Trial 42 finished with value: 0.7826210826210825 and parameters: {'learning_rate': 0.13215802743760421, 'n_estimators': 54, 'subsample': 0.956231925615068, 'max_depth': 4}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-24 17:21:14,814] Trial 44 finished with value: 0.7903133903133902 and parameters: {'learning_rate': 0.12244889990030039, 'n_estimators': 51, 'subsample': 0.9404509240168005, 'max_depth': 4}. Best is trial 2 with value: 0.8132478632478632.
[I

Best trial: 2. Best value: 0.813248:  96%|█████████▌| 48/50 [00:10<00:00,  5.95it/s]

[I 2025-12-24 17:21:14,951] Trial 48 finished with value: 0.7941595441595442 and parameters: {'learning_rate': 0.12302702489179099, 'n_estimators': 74, 'subsample': 0.9189466839438173, 'max_depth': 3}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-24 17:21:14,998] Trial 47 finished with value: 0.7941595441595442 and parameters: {'learning_rate': 0.1242413761625736, 'n_estimators': 51, 'subsample': 0.9412472282717741, 'max_depth': 3}. Best is trial 2 with value: 0.8132478632478632.


Best trial: 2. Best value: 0.813248: 100%|██████████| 50/50 [00:10<00:00,  4.64it/s]

[I 2025-12-24 17:21:15,334] Trial 49 finished with value: 0.8014245014245015 and parameters: {'learning_rate': 0.12536320295042022, 'n_estimators': 74, 'subsample': 0.9364434168459242, 'max_depth': 7}. Best is trial 2 with value: 0.8132478632478632.


In [27]:
best_params_meta: dict = study_gbm.best_params.copy()
META_MODEL: GradientBoostingClassifier = GradientBoostingClassifier(
    **best_params_meta , random_state = int(13))
META_MODEL.fit(META_X, META_Y)


,loss,'log_loss'
,learning_rate,0.13207309340036216
,n_estimators,31
,subsample,0.8870897857825569
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [28]:
from sklearn.linear_model import LinearRegression
Meta_2_model: LinearRegression = LinearRegression()
Meta_2_model.fit(META_X , META_Y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [29]:
test_df: pd.DataFrame = pd.read_csv('test.csv')
test_df.columns = [
    'date',
    'temp_9am',
    'temp_3pm',
    'min_temp',
    'max_temp',
    'rain_fall',
    'evaporation',
    'sunshine',
    'wind_gust_dir',
    'wind_gust_speed',
    'wind_dir_9am',
    'wind_dir_3pm',
    'wind_speed_9am',
    'wind_speed_3pm',
    'humidity_9am',
    'humidity3pm',
    'pressure_9am',
    'pressure_3pm',
    'cloud_9am',
    'cloud_3pm',
    'id'
]
ids = test_df['id']
test_df = test_df.drop('id' , axis = 1)
test_df['date'] = test_df['date'].map(date_to_days)


In [30]:
cat_columns: list = []
for item in test_df.columns:
    if test_df[item].dtype == object:
        cat_columns.append(item)
print(cat_columns)
for item in cat_columns: print(len(test_df[item].unique()))

['wind_gust_dir', 'wind_dir_9am', 'wind_dir_3pm']
16
16
16


In [31]:
encoder = OneHotEncoder(sparse_output = False , 
                        categories = 'auto', 
                        handle_unknown= 'error',
                        max_categories = 32)

encoder.fit(test_df[cat_columns])
cat_ohed = pd.DataFrame(encoder.transform(test_df[cat_columns]))

In [32]:
test_df = test_df.drop(cat_columns, axis = 1)
test_df = pd.concat([test_df , cat_ohed], axis = 1)
test_df

,date,temp_9am,temp_3pm,min_temp,max_temp,rain_fall,evaporation,sunshine,wind_gust_speed,wind_speed_9am,...,38,39,40,41,42,43,44,45,46,47
0,7396,12.7,17.0,9.0,19.7,0.0,3.2,10.8,31,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7397,11.9,18.6,8.0,18.8,0.0,4.0,10.4,31,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7398,14.4,20.6,11.2,20.8,2.8,5.6,8.0,52,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,7399,13.7,18.3,12.5,19.5,20.4,4.0,5.7,48,19,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7400,15.2,18.7,11.5,21.7,0.0,2.8,9.3,37,17,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,8046,10.5,17.9,8.6,19.6,0.0,2.0,7.8,37,22,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
651,8047,11.0,18.7,9.3,19.2,0.0,2.0,9.2,30,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
652,8048,10.2,17.3,9.4,17.7,0.0,2.4,2.7,24,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
653,8049,12.4,19.0,10.1,19.3,0.0,1.4,9.3,43,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
base_xgbm_test_pred: np.ndarray = BASE_MODEL_XGBM.predict_proba(test_df)
base_lgbm_test_pred: np.ndarray = BASE_MODEL_LGBM.predict_proba(test_df)
base_cb_test_pred  : np.ndarray = BASE_CB_MODEL.predict_proba(test_df)

meta_test_df: pd.DataFrame =  pd.DataFrame({
    "XGBM_PREDICTION" : base_xgbm_test_pred[:, 1],
    "LGBM_PREDICTION" : base_lgbm_test_pred[:, 1],
    "CB_PREDICTION"   : base_cb_test_pred[:, 1]
})

In [34]:
final_result: np.ndarray = META_MODEL.predict(meta_test_df)
final_result

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [35]:
res: int = final_result.sum()/len(final_result)
res

np.float64(0.16030534351145037)

In [41]:
resultDataFrame: pd.DataFrame = pd.DataFrame({
    'ID' : ids,
    'Predicted' : pd.Series(final_result).map({1 : "yes" , 0 : "no"})
})

In [42]:
resultDataFrame.to_csv("result.csv" , index = False)